## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2024">The Repo</a>.  If you are unclear on how to pull an updated copy using the GitHub command line, the following <a href="https://techwritingmatters.com/how-to-update-your-forked-repository-on-github">document</a> is helpful.  Be sure to add the professors and TAs as collaborators on your project. 

- lpalum@gmail.com GitHub ID: lpalum
- ajay.anand@rochester.edu GitHub ID: ajayan12
- divyamunot1999@gmail.com GitHub ID: divyamunot
- ylong6@u.Rochester.edu GitHub ID: NinaLong2077

Once you have updates your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://www.databricks.training/step-by-step/importing-courseware-from-github/index.html">Repos</a> feature.
Each student is expected to submit the URL of their project on GitHub with their code checked in on the main/master branch.  This illustration highlights the branching scheme that you may use to work on your code in steps and then merge your submission into your master branch before submitting.
</p>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/github.drawio.png">
<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches.  First, you may choose to start the bronze_stream and let it complete (read and append all of the source data) before preceeding and starting up the silver_stream.  This approach has latency associated with it but it will allow your code to proceed in a linear fashion and process all the data by the end of your notebook execution.  Another potential approach is to start all the streams and have a "watch" method to determine when the pipeline has processed sufficient or all of the source data before stopping and displaying results.  Both of these approaches are valid and have different implications on how you will trigger your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)

### Be sure that your project runs end to end when *Run all* is executued on this notebook! (15 Points out of 60)

In [0]:
%run ./includes/includes

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

from delta import *
dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, BRONZE_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, SILVER_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, GOLD_DELTA).optimize().executeCompaction()
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here...
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

Since the cluster has four cores, I've kept the shuffle partitions at the default of 200, which is a multiple of four.

In [0]:
# Importing pyspark sql types and functions to perform neccessary transformations
from pyspark.sql.types import ArrayType, DoubleType, IntegerType, LongType, StringType, StructType, StructField, TimestampType, MapType
from pyspark.sql.functions import input_file_name, current_timestamp, col, split, explode, regexp_replace, substring, to_timestamp, udf, when, countDistinct, count, sum, min, max
from pyspark.sql import Window

# Importing libraries to create model, get accuracy metrics, and log to mlflow
from transformers import pipeline
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import mlflow
import mlflow.pyfunc
import cloudpickle

# Importing seaborn and seaborn objects for graphing
# Note: Include file has been modified to update seaborn for using the seaborn objects API. This is my preferred graphing library
import seaborn as sns
import seaborn.objects as so

In [0]:
# Log time to import libraries
LIBRARY_TIME = time.time()

## 2.0 Use the utility functions to ...
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
# Use utility functions to get source files list, count number of files, and show example file
src_listing_df = get_source_listing_df()
file_count = src_listing_df['File Name'].count()
ex_file = src_listing_df['File Name'][1]
ex_file_name = src_listing_df['File Name'][1].split("/")[1]
ex_file_text = show_s3_file_contents(ex_file).decode('utf-8')

print(f"Number of files in source directory: \n {file_count} \n")
print(f"Contents of {ex_file_name}: \n {ex_file_text}")

In [0]:
# Time utility functions
UTILITY_TIME = time.time()

## 3.0 Transform the Raw Data to Bronze Data using a stream
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using cloudfiles to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defines in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
# Create struct of raw schema
raw_schema = StructType([
    StructField("date", StringType(), True),
    StructField("user", StringType(), True),
    StructField("text", StringType(), True),
    StructField("sentiment", StringType(), True)
])

In [0]:
# Set up raw stream using cloudfiles with schema enforced
raw_stream = (
    spark
    .readStream
    .format("cloudfiles")
    .option("cloudfiles.format","json")
    .schema(raw_schema)
    .load(TWEET_SOURCE_PATH)
    )

In [0]:
# Transform raw stream to comply with directions: Source file and timestamp added
raw_transform = (
    raw_stream
    .withColumn('source_file',input_file_name())
    .withColumn('processed_time',current_timestamp())
)

In [0]:
# Write stream to bronze table, mergeSchema enabled
bronze_query = (
    raw_transform
    .writeStream
    .option("mergeSchema","true")
    .outputMode("append")
    .format('delta')
    .queryName("bronze_stream")
    .option("checkpointLocation",BRONZE_CHECKPOINT)
    .start(BRONZE_DELTA)
)

In [0]:
# Modified utility function to wait for stream to start by checking that rcent progress has been updated three times
# Get total time to confirmation of bronze stream
bronze_started = wait_stream_start(spark,"bronze_stream")
BRONZE_TIME = time.time()

## 4.0 Bronze Data Exploratory Data Analysis
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
# Read bronze table for EDA
bronze_eda = (spark.read.format("delta").load(BRONZE_DELTA))

In [0]:
# Count number of tweets in bronze table
# Note: 200,000 expected when completed
tweet_count = bronze_eda.count()
print(f"Number of tweets in Bronze Table: \n {tweet_count}")

In [0]:
# Get null values in bronze table columns
# Note: none found
bronze_nulls = bronze_eda.select([count(when(col(c).isNull(),c)).alias(c) for c in bronze_eda.columns]).collect()[0].asDict()
for fld in bronze_nulls:
    print(f"Column '{fld}' has {bronze_nulls[fld]} missing values.")

In [0]:
# Get top 20 usernames by number of tweets
user_tweet_counts = bronze_eda.groupBy("user").count().sort("count",ascending=False).limit(20)
user_tweet_counts.show()

In [0]:
# Find tweets with a mention by looking for '@' symbol
# Note: 93,817 with at least one mention when completed

mention_count = bronze_eda.where(bronze_eda.text.like('%@%')).count()
no_mention_count = bronze_eda.where(~ bronze_eda.text.like('%@%')).count()

print(f"{mention_count} tweets have at least one mention.")
print(f"{no_mention_count} tweets have no mentions.")

In [0]:
# Plot top 20 tweeters

p = (
    so.Plot(
        data = user_tweet_counts.toPandas().iloc[:20,:],
        x = "user",
        y= "count"
    )
    .add(so.Bar())
    .label(
        x = "Username",
        y = "Tweet Count",
        title = "Top 20 Tweeters"
    )
    .plot()
)

p._figure.axes[0].xaxis.set_tick_params(rotation = 90)

p

In [0]:
# Get time for EDA
EDA_TIME = time.time()

## 5.0 Transform the Bronze Data to Silver Data using a stream
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
# Set up stream reader from bronze to silver
# Found good results setting max files per trigger to 100, kept pace with bronze table
bronze_to_silver_read = (
    spark
    .readStream
    .option("maxFilesPerTrigger",100)
    .format('delta')
    .load(BRONZE_DELTA)
)

In [0]:
# Transform stream for silver schema. Convert time stamp, extract mentions from tweet text, and clean text without mentions
bronze_to_silver_tr = (
    bronze_to_silver_read
    .withColumn("timestamp",to_timestamp(substring(col("date"),5,25),"MMM dd kk:mm:ss zzz yyyy"))
    .withColumn('words',split(col('text')," "))
    .withColumn('mention',explode(split(col('text')," ")))
    .filter("mention like '@%' and mention <> '@'")
    .withColumn("cleaned_text",regexp_replace(col('text'),"(\s|^)@\w+",""))
    .select("timestamp","mention","cleaned_text","sentiment")
)

In [0]:
# Write stream to Silver table, mergeSchema allowed
bronze_to_silver_query = (
    bronze_to_silver_tr
    .writeStream
    .option("mergeSchema","true")
    .outputMode("append")
    .format('delta')
    .queryName("silver_stream")
    .option("checkpointLocation",SILVER_CHECKPOINT)
    .start(SILVER_DELTA)
)

In [0]:
# Wait for silver stream to start, get time
silver_started = wait_stream_start(spark, "silver_stream")
SILVER_TIME = time.time()

## 6.0 Transform the Silver Data to Gold Data using a stream
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
# Set up stream reader from silver to gold
# Found good results setting max files per trigger to 10
silver_to_gold_read = (
    spark
    .readStream
    .option("maxFilesPerTrigger",10)
    .format('delta')
    .load(SILVER_DELTA)
)

In [0]:
# Import model and create pandas UDF
# Perform inference on tweets
model_version_uri = f"models:/{MODEL_NAME}/production"
HF_UDF = mlflow.pyfunc.spark_udf(spark,model_uri=model_version_uri)

silver_to_gold_tr = (
    silver_to_gold_read
    .withColumn("predicted",HF_UDF(col("cleaned_text")))
    .withColumn("predicted_sentiment",when(col("predicted")['label']=='POS',"positive").when(col("predicted")['label']=='NEG',"negative").when(col("predicted")['label']=='NEU',"neutral"))
    .withColumn("predicted_score",(col('predicted')['score']).astype(DoubleType()))
    .drop("predicted")
    .withColumn("sentiment_id",when(col('sentiment') == 'positive',1).when(col('sentiment') == 'negative',0))
    .withColumn("predicted_sentiment_id",when(col('predicted_sentiment') == 'positive',1).when(col('predicted_sentiment') == 'negative',0))
)

In [0]:
# Write results to Gold table
silver_to_gold_query = (
    silver_to_gold_tr
    .writeStream
    .option("mergeSchema","True")
    .outputMode("append")
    .format('delta')
    .queryName("gold_stream")
    .option("checkpointLocation",GOLD_CHECKPOINT)
    .start(GOLD_DELTA)
)

In [0]:
# Wait for gold to start and get time
gold_started = wait_stream_start(spark, "gold_stream")
GOLD_TIME = time.time()

## 7.0 Capture the accuracy metrics from the gold table in MLflow
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the mdoel name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:
# Read gold table
gld = (
    spark
    .read
    .format("delta")
    .load(GOLD_DELTA)
)

In [0]:
# Get sentiment ID and predicted sentiment ID
# Note: Predicted sentiment ID is null when predicted sentiment is neutral, so those are not used for these metrics
# Get Precision, Recall and F! scores and create a confusion matrix
labels = (
    gld
    .select("sentiment_id","predicted_sentiment_id")
    .toPandas()
    .query("~predicted_sentiment_id.isnull()")
)

prec = precision_score(labels['sentiment_id'],labels['predicted_sentiment_id'].astype(int))

recall = recall_score(labels['sentiment_id'],labels['predicted_sentiment_id'].astype(int))

f1 = f1_score(labels['sentiment_id'],labels['predicted_sentiment_id'].astype(int))

conf = confusion_matrix(labels['sentiment_id'],labels['predicted_sentiment_id'].astype(int))

print(f"Pecision score : {prec}")
print(f"Recall score : {recall}")
print(f"F1 score : {f1}")

In [0]:
# Create heatmap from confusion matrix

hmap = sns.heatmap(conf)
hmap.get_figure().savefig('conf.png')
hmap.plot()

In [0]:
# Get latest version of silver table
slv_version = spark.sql("select max(version) from (describe history SILVER_DELTA)").collect()[0][0]

In [0]:
# Log metrics, artifact and parameter
with mlflow.start_run(run_name="TWalton_Final_Log") as run:
    mlflow.log_metric('Precision',prec)
    mlflow.log_metric('Recall',recall)
    mlflow.log_metric('F1',f1)

    mlflow.log_artifact('conf.png')

    mlflow.log_param('Silver_version',slv_version)

In [0]:
# Get time for metrics and logging
ML_FLOW_TIME = time.time()

## 8.0 Application Data Processing and Visualization
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

You may want to use the "Loop Application" widget to control whether you repeateded display the latest plots while the data comes in from your streams before moving on to the next section and cleaning up your run.

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
# Read gold table
# Redundant after above, but kept in case processed out of order
gld = (
    spark
    .read
    .format("delta")
    .load(GOLD_DELTA)
)

In [0]:
# Create app data table by counting tweets of each sentiment for each mentioned user

app_data = (
    gld
    .withColumn(
        "min_timestamp",
        min(col("timestamp")).over(Window.partitionBy(["mention"]))
    )
    .withColumn(
        "max_timestamp",
        max(col("timestamp")).over(Window.partitionBy(["mention"]))
    )
    .withColumn(
        "mention_count",
        count(col("cleaned_text")).over(Window.partitionBy(["mention","predicted_sentiment"]))
    )
    .select("min_timestamp","max_timestamp","mention","predicted_sentiment","mention_count")
    .drop_duplicates()
    .groupBy("min_timestamp","max_timestamp","mention")
    .pivot("predicted_sentiment",["negative","neutral","positive"])
    .sum()
    .fillna(0,["negative","neutral","positive"])
    .withColumn(
        "total",
        (col("negative")+col("neutral")+col("positive"))
    )
    .sort("total",ascending=False)
)

In [0]:
# Display App data
display(app_data)

In [0]:
# Count distinct mentions captured in gold table

distinct_mentions_count = app_data.count()
print(f"Number of distinct mentions in Gold Table: \n {distinct_mentions_count}")

In [0]:
# Get top 20 users mentioned in positive tweets

top_20_pos = (
    app_data
    .select('mention','positive')
    .sort('positive',ascending=False)
    .limit(20)
)
display(top_20_pos)

In [0]:
# Get top 20 users mentioned in negative tweets

top_20_neg = (
    app_data
    .select('mention','negative')
    .sort('negative',ascending=False)
    .limit(20)
)
display(top_20_neg)

In [0]:
# Plot top 20 users mentioned in positive tweets

pos_20 = (
    so.Plot(
        data = top_20_pos.toPandas().iloc[:20,:],
        x = "mention",
        y= "positive"
    )
    .add(so.Bar())
    .label(
        x = "Username",
        y = "Positive Mention Count",
        title = "Top 20 Positively Mentioned"
    )
    .plot()
)

pos_20._figure.axes[0].xaxis.set_tick_params(rotation = 90)

pos_20

In [0]:
# Plot top 20 users mentioned in negative tweets

neg_20 = (
    so.Plot(
        data = top_20_neg.toPandas().iloc[:20,:],
        x = "mention",
        y= "negative"
    )
    .add(so.Bar())
    .label(
        x = "Username",
        y = "Negative Mention Count",
        title = "Top 20 Negatively Mentioned"
    )
    .plot()
)

neg_20._figure.axes[0].xaxis.set_tick_params(rotation = 90)

neg_20

In [0]:
# Get time for Application data

APP_TIME = time.time()

## 9.0 Clean up and completion of your pipeline
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook.

In [0]:
# Print all streams still active
print("Active streams: ")
for stream in spark.streams.active:
    print(stream.name)

In [0]:
# Either stops all active streams or only stops streams that are no longer processing files. 
# Created the option here so that the notebook can be run completely but still allow streams to keep running if desired, but so that these streams won't keep spinning if finished
# Will default to the former upon submission, but welcome to set STOP_ALL_STREAMS to False to see how it runs when the tables are deleted.

STOP_ALL_STREAMS = True

if STOP_ALL_STREAMS:
    stop_all_streams()
else:
    for stream in spark.streams.active:
        row_total = 0
        for prog in stream.recentProgress[-5:]:
            row_total = row_total + prog['numInputRows']
        if row_total == 0:
            stream.stop()

In [0]:
# Get the notebooks ending time note START_TIME was established in the include file when the notebook started.
END_TIME = time.time()

In [0]:
library_min, library_sec = divmod(LIBRARY_TIME-START_TIME,60)
print(f"Time Elapsed Importing Libraries: \n {library_min} Minutes, {library_sec} Seconds")

utility_min, utility_sec = divmod(UTILITY_TIME-LIBRARY_TIME,60)
print(f"Time Elapsed Checking Directory: \n {utility_min} Minutes, {utility_sec} Seconds \n")

bronze_min, bronze_sec = divmod(BRONZE_TIME-UTILITY_TIME,60)
print(f"Time Elapsed Starting Bronze Stream: \n {bronze_min} Minutes, {bronze_sec} Seconds \n")

EDA_min, EDA_sec = divmod(EDA_TIME-BRONZE_TIME,60)
print(f"Time Elapsed on Bronze EDA: \n {EDA_min} Minutes, {EDA_sec} Seconds \n")

silver_min, silver_sec = divmod(SILVER_TIME-EDA_TIME,60)
print(f"Time Elapsed Starting Silver Stream: \n {silver_min} Minutes, {silver_sec} Seconds \n")

gold_min, gold_sec = divmod(GOLD_TIME-SILVER_TIME,60)
print(f"Time Elapsed Starting Gold Stream: \n {gold_min} Minutes, {gold_sec} Seconds \n")

mlflow_min, mlflow_sec = divmod(ML_FLOW_TIME-GOLD_TIME,60)
print(f"Time Elapsed Logging MLFlow Data: \n {mlflow_min} Minutes, {mlflow_sec} Seconds \n")

app_min, app_sec = divmod(APP_TIME-ML_FLOW_TIME,60)
print(f"Time Elapsed Creating App Data: \n {app_min} Minutes, {app_sec} Seconds \n")

Total_Time_min, Total_Time_sec = divmod(END_TIME-START_TIME,60)
print(f"Total Time Elapsed: \n {Total_Time_min} Minutes, {Total_Time_sec} Seconds")


## 10.0 How Optimized is your Spark Application (Grad Students Only)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

### Spill
The data stored on disk was approximately 0%, indicating little to no spill.
### Skew
I believe keeping the partion number high and reducing files per trigger, this prevented skew by allocating partitions to the streams that needed them most. The gold stream typically had fewer tasks per job, typically only 4, but the bronze stream would sometimes have over 30 per job.
### Shuffle
Shuffle was kept low, with total shuffle read and write coming to less than a third of a GiB.
### Storage
Storage was kept to a low percentage of overall memory, some number of MiB out of 21 GiB.
### Serialization
Using the pandas UDF was critical in reducing serialization. As an example I attempted to construct the gold table without it and the stream was unable to process at nearly the same speed.